## Introduction

In this notebook, cattle movement during the day and the night is visualised using an interactive plot using Leafmap with Lonboard at the backend.

The data have been prepared in another script as the raw data from Movebank are in the form of a .csv file and the data need to be converted to a Geopandas dataframe for further analysis.

This code uses the [cattle data from Movebank of [Moritz et al.](https://ecodiv.earth/post/animal_tracking-data/#ref-moritz2018)](https://www.movebank.org/cms/webapp?gwt_fragment=page=studies,path=study412206724) [1][2] and makes an interactive plot using the package [Leafmap with Deckgl](https://leafmap.org/deckgl/#leafmap.deckgl.apply_continuous_cmap) at the backend. This package is developed by [Quisheng Wu](https://wetlands.io/).


The motivation for this code is that during Day 5 of the [30DayMapChallenge](https://30daymapchallenge.com/) where the topic was "movement", I decided to study movement of animals and use the amazing Movebank dataset. There is a wealth of data on the Movebank website and quite a lot of data you can download for free.

I use the same data as the excellent QGIS tutorial of [Ecodiv.earth](https://ecodiv.earth/post/animal_tracking-data/#ref-moritz2018) [3]. Inititially (having only one day of time), I struggled getting my way around in Movebank as I was overwhelmed with the choices and new to exploring. The only time I had downloaded some data of Movebank is for a book review using some data from Movebank (on cougars).

Initially, I aimed for the sky and wanted to create an animated visualisation in Folium (since I am new to animation in Python). After a few lines of code in Folium, I realised that Folium crashed. I was wondering why just visualising the data just crashed.

At the time of writing, I was also listening to the daily YouTube videos of Ujaval Gandhi's course on [Mapping & Data visualisation in Python](https://www.youtube.com/playlist?list=PLppGmFLhQ1HLzGl8auwYkdUMu_z0Hz7G6) [3][4] and he mentioned that Folium will crash. It is then that I realised that there is something called "Lonboard" developed by [Development Seed](https://developmentseed.org/lonboard/latest/) and that Leafmap interacts with Lonboard. Added to this, the Leafmap lectures of Quisheng Wu in his "Intro to Programming" course are also super useful and amazing for understanding the basics of Leafmap [5].  

The script below is still very  basic but at least I realised that there are tremendous resources out there in the form of datasets ([Movebank](https://www.movebank.org/cms/movebank-main), software people have written [Leafmap](https://leafmap.org/) and [Lonboard]() and tutorials ([Spatial Thoughts](https://spatialthoughts.com/) and [Quisheng Wu](https://www.youtube.com/@giswqs) and that I have just scratched the surface!

As pointed out in the video lectures of Spatial Thoughts, the syntax for creating the maps differs whether you use points, lines, etc. Unlike the tutorial of Ujaval Gandhi that uses lines (rivers), this Movebank dataset has points. For points, one needs to refer to the [Lonboard documentation for points](https://developmentseed.org/lonboard/latest/api/layers/scatterplot-layer/#lonboard.ScatterplotLayer).

This code is very basic as of now so comments are welcome (through Github issues) or other. Thank you for reading!

## Installing packages

In [1]:
!pip install leafmap lonboard --quiet

In [2]:
import os
import leafmap.deckgl as leafmap
import geopandas as gpd
import pandas as pd
import requests
import lonboard

The data are downloaded from Movebank and downloaded on my hard disk. Subsequently, the data are uploaded in Google Colab but if you use Leafmap on your local computer or elsewhere, you have to change the path in the next cell. When uploading the data in Google Colab, please do not forget to upload all the files (.cpg, .dbf, .prj and .shx) next to the .shp file as otherwise the file will not read properly.

In [3]:
df = gpd.read_file('/content/movement.shp')

In [4]:
df.shape

(166921, 17)

In [5]:
df.shape

(166921, 17)

In [6]:
df.head()

,event-id,visible,timestamp,location-l,location_1,comments,ground-spe,heading,height-abo,study-spec,tag-tech-s,sensor-typ,individual,tag-local-,individu_1,study-name,geometry
0,5170988886,True,2009-03-12 05:11:00.000,15.088783,11.146467,day,0.044704,243.0,302.9712,4-26,0,gps,Bos taurus,cattle0120309day,cattle0120309day,Daily grazing movements of cattle in the Far N...,POINT (15.08878 11.14647)
1,5170988887,True,2009-03-12 05:11:00.000,15.088783,11.146450,day,0.625856,289.0,305.1048,6-3,1,gps,Bos taurus,cattle0120309day,cattle0120309day,Daily grazing movements of cattle in the Far N...,POINT (15.08878 11.14645)
2,5170988888,True,2009-03-12 05:11:00.000,15.088767,11.146467,day,0.894080,304.0,305.1048,7-3,2,gps,Bos taurus,cattle0120309day,cattle0120309day,Daily grazing movements of cattle in the Far N...,POINT (15.08877 11.14647)
3,5170988889,True,2009-03-12 05:11:00.000,15.088750,11.146467,day,1.788160,306.0,306.0192,17-3,3,gps,Bos taurus,cattle0120309day,cattle0120309day,Daily grazing movements of cattle in the Far N...,POINT (15.08875 11.14647)
4,5170988890,True,2009-03-12 05:11:00.000,15.088700,11.146500,day,0.894080,297.0,313.0296,9-3,4,gps,Bos taurus,cattle0120309day,cattle0120309day,Daily grazing movements of cattle in the Far N...,POINT (15.0887 11.1465)


Next, the data for day and night is split up. When the cattle does not move, there is no observation.

Obtaining the cattle in the night:

In [7]:
df_night = df[df['comments']=='night']

In [8]:
df_night.head()

,event-id,visible,timestamp,location-l,location_1,comments,ground-spe,heading,height-abo,study-spec,tag-tech-s,sensor-typ,individual,tag-local-,individu_1,study-name,geometry
98956,5171087856,True,2009-03-12 18:00:00.000,15.088900,11.146467,night,2.235200,80.0,316.0776,22-3,0,gps,Bos taurus,cattle2120309night,cattle2120309night,Daily grazing movements of cattle in the Far N...,POINT (15.0889 11.14647)
98957,5171087857,True,2009-03-12 18:00:00.000,15.088950,11.146483,night,0.044704,270.0,316.0776,6-38,1,gps,Bos taurus,cattle2120309night,cattle2120309night,Daily grazing movements of cattle in the Far N...,POINT (15.08895 11.14648)
98958,5171087858,True,2009-03-12 18:00:00.000,15.088933,11.146483,night,0.447040,71.0,317.9064,6-4,2,gps,Bos taurus,cattle2120309night,cattle2120309night,Daily grazing movements of cattle in the Far N...,POINT (15.08893 11.14648)
98959,5171087859,True,2009-03-12 18:00:00.000,15.088950,11.146483,night,0.581152,0.0,317.9064,6-3,3,gps,Bos taurus,cattle2120309night,cattle2120309night,Daily grazing movements of cattle in the Far N...,POINT (15.08895 11.14648)
98960,5171087860,True,2009-03-12 18:01:00.000,15.088950,11.146500,night,0.044704,316.0,317.9064,3-13,4,gps,Bos taurus,cattle2120309night,cattle2120309night,Daily grazing movements of cattle in the Far N...,POINT (15.08895 11.1465)


In [9]:
counts_cattle_night = df_night['tag-local-'].value_counts()

In [10]:
print(f'There are {len(counts_cattle_night)} cattle observed in the night')

There are 12 cattle observed in the night


Obtaining the cattle in the day:

In [11]:
df_day = df[df['comments']=='day']

In [12]:
df_day.head()

,event-id,visible,timestamp,location-l,location_1,comments,ground-spe,heading,height-abo,study-spec,tag-tech-s,sensor-typ,individual,tag-local-,individu_1,study-name,geometry
0,5170988886,True,2009-03-12 05:11:00.000,15.088783,11.146467,day,0.044704,243.0,302.9712,4-26,0,gps,Bos taurus,cattle0120309day,cattle0120309day,Daily grazing movements of cattle in the Far N...,POINT (15.08878 11.14647)
1,5170988887,True,2009-03-12 05:11:00.000,15.088783,11.146450,day,0.625856,289.0,305.1048,6-3,1,gps,Bos taurus,cattle0120309day,cattle0120309day,Daily grazing movements of cattle in the Far N...,POINT (15.08878 11.14645)
2,5170988888,True,2009-03-12 05:11:00.000,15.088767,11.146467,day,0.894080,304.0,305.1048,7-3,2,gps,Bos taurus,cattle0120309day,cattle0120309day,Daily grazing movements of cattle in the Far N...,POINT (15.08877 11.14647)
3,5170988889,True,2009-03-12 05:11:00.000,15.088750,11.146467,day,1.788160,306.0,306.0192,17-3,3,gps,Bos taurus,cattle0120309day,cattle0120309day,Daily grazing movements of cattle in the Far N...,POINT (15.08875 11.14647)
4,5170988890,True,2009-03-12 05:11:00.000,15.088700,11.146500,day,0.894080,297.0,313.0296,9-3,4,gps,Bos taurus,cattle0120309day,cattle0120309day,Daily grazing movements of cattle in the Far N...,POINT (15.0887 11.1465)


In [13]:
df_day.columns

Index(['event-id', 'visible', 'timestamp', 'location-l', 'location_1',
       'comments', 'ground-spe', 'heading', 'height-abo', 'study-spec',
       'tag-tech-s', 'sensor-typ', 'individual', 'tag-local-', 'individu_1',
       'study-name', 'geometry'],
      dtype='object')

See how many cattle are there during the day:

In [14]:
counts_cattle_day = df_day['tag-local-'].value_counts()

In [15]:
print(f'There are {len(counts_cattle_day)} cattle observed during the day')

There are 21 cattle observed during the day


In [16]:
df_day.head()

,event-id,visible,timestamp,location-l,location_1,comments,ground-spe,heading,height-abo,study-spec,tag-tech-s,sensor-typ,individual,tag-local-,individu_1,study-name,geometry
0,5170988886,True,2009-03-12 05:11:00.000,15.088783,11.146467,day,0.044704,243.0,302.9712,4-26,0,gps,Bos taurus,cattle0120309day,cattle0120309day,Daily grazing movements of cattle in the Far N...,POINT (15.08878 11.14647)
1,5170988887,True,2009-03-12 05:11:00.000,15.088783,11.146450,day,0.625856,289.0,305.1048,6-3,1,gps,Bos taurus,cattle0120309day,cattle0120309day,Daily grazing movements of cattle in the Far N...,POINT (15.08878 11.14645)
2,5170988888,True,2009-03-12 05:11:00.000,15.088767,11.146467,day,0.894080,304.0,305.1048,7-3,2,gps,Bos taurus,cattle0120309day,cattle0120309day,Daily grazing movements of cattle in the Far N...,POINT (15.08877 11.14647)
3,5170988889,True,2009-03-12 05:11:00.000,15.088750,11.146467,day,1.788160,306.0,306.0192,17-3,3,gps,Bos taurus,cattle0120309day,cattle0120309day,Daily grazing movements of cattle in the Far N...,POINT (15.08875 11.14647)
4,5170988890,True,2009-03-12 05:11:00.000,15.088700,11.146500,day,0.894080,297.0,313.0296,9-3,4,gps,Bos taurus,cattle0120309day,cattle0120309day,Daily grazing movements of cattle in the Far N...,POINT (15.0887 11.1465)


## Analysis for the day observations

In [17]:
df_day.columns

Index(['event-id', 'visible', 'timestamp', 'location-l', 'location_1',
       'comments', 'ground-spe', 'heading', 'height-abo', 'study-spec',
       'tag-tech-s', 'sensor-typ', 'individual', 'tag-local-', 'individu_1',
       'study-name', 'geometry'],
      dtype='object')

In [18]:
df_day.dtypes

,0
event-id,int64
visible,bool
timestamp,object
location-l,float64
location_1,float64
comments,object
ground-spe,float64
heading,float64
height-abo,float64
study-spec,object


Only keep the relevant data:

In [19]:
df_day = df_day[['timestamp', 'tag-local-', 'geometry']]

The unique values of the cattle:

In [20]:
unique_values = df_day['tag-local-'].unique()
print(unique_values)

['cattle0120309day' 'cattle0130309day' 'cattle0150309day'
 'cattle0160309day' 'cattle1110309day' 'cattle1120309day'
 'cattle1130309day' 'cattle1150309day' 'cattle1160309day'
 'cattle1170309day' 'cattle1230309day' 'cattle1240309day'
 'cattle1250309day' 'cattle2110309day' 'cattle2150309day'
 'cattle2170309day' 'cattle2230309day' 'cattle2240309day'
 'cattle3110309day' 'cattlex240309day' 'cattlex250309day']


In [21]:
len(unique_values)

21

Initially I got stuck here as I wanted to have a different color for each cow. There are 21 cows in the day time dataset and this is where I had to start deviation of the original tutorial on the river dataset of Ujaval Gandhi since I had many more categories (21 instead of 10) and my identifiers/tags are in string format. Initially, I wanted to follow the string format mapping following the [document of Lonboard](https://developmentseed.org/lonboard/latest/api/colormap/) but this threw an error so I must be doing something wrong and hope to figure this out soon.

Also, I could have used the "pallettable" library (like in the SpatialThoughts tutorial) but I had just too many classes (21).

Instead I manually assigned integers to each cattle identifier ('cattelxxxx').

First, I create a unique number for each cattle:

In [22]:
def categorise(tag):
  for i in range(len(unique_values)):
    if tag == unique_values[i]:
      return i


And then the ultimate mapping:

In [23]:
# Apply the function to the Age column using the apply() function which I also name "color"
df_day['color'] = df_day['tag-local-'].apply(categorise)

In [24]:
df_day.head()

,timestamp,tag-local-,geometry,color
0,2009-03-12 05:11:00.000,cattle0120309day,POINT (15.08878 11.14647),0
1,2009-03-12 05:11:00.000,cattle0120309day,POINT (15.08878 11.14645),0
2,2009-03-12 05:11:00.000,cattle0120309day,POINT (15.08877 11.14647),0
3,2009-03-12 05:11:00.000,cattle0120309day,POINT (15.08875 11.14647),0
4,2009-03-12 05:11:00.000,cattle0120309day,POINT (15.0887 11.1465),0


In [25]:
df_day.color.dtype

dtype('int64')

Change to integer:

In [26]:
df_day['color'] = df_day['color'].astype(int)

In [27]:
df_day.dtypes

,0
timestamp,object
tag-local-,object
geometry,geometry
color,int64


In order to manually assign categories, I actually combined the color maps using the color mapping of the ESRI Global Land Cover and  Dynamic World Land Cover from the Quisheng Wu's tutorials for Geemap [6].

I pasted the two color maps together and created a random color combination for the 21th animal:

In [28]:
colormap = {
  0: [26, 91, 171],
  1: [53, 130, 33],
  2: [167, 210, 130],
  3: [135, 209, 158],
  4: [255, 219, 92] ,
  5: [238, 207, 168],
  6: [237, 2, 42],
  7: [237, 233, 228],
  8: [242, 250, 255],
  9: [200, 200, 200],
  10: [65, 155, 223],
  11: [57, 125, 73],
  12: [136, 176, 83],
  13: [122, 135, 198],
  14: [228, 150, 53],
  15: [223, 195, 90],
  16: [196, 40, 27] ,
  17: [165, 155, 143],
  18: [179, 159, 225],
  19: [65, 155, 223 ],
  20: [66, 100, 191]
}

In [29]:
len(colormap)

21

From here, I continue to follow the code of Spatial Thoughts [3] for the river dataset.

In [30]:
colors = lonboard.colormap.apply_categorical_cmap(df_day['color'], colormap)

In [31]:
colors

array([[ 26,  91, 171],
       [ 26,  91, 171],
       [ 26,  91, 171],
       ...,
       [ 66, 100, 191],
       [ 66, 100, 191],
       [ 66, 100, 191]], dtype=uint8)

Since these are day time observations, I decided to take a light background. To start with, let us not play different colors for the animals and just color the lines with blue.

Please note also that here the syntax differs from the original tutorial which uses lines (for rivers of Spatial Thoughts) instead of points here. For points, you have to use "get_fill_color" while for lines you have to use "get_color"

In [33]:
m = leafmap.Map(height=600)
m.add_basemap('CartoDB.Positron')
data = df_day
m.add_gdf(data, get_fill_color = 'blue')
# m #PLEASE UNCOMMENT THIS AS I COMMENTED THIS AS GITHUB RENDERING IS STRUGGLING OTHERWISE. SCREENSHOT OF MAP IS BELOW.

From here, I could continue follow the code of Spatial Thoughts:

In [34]:
colors = lonboard.colormap.apply_categorical_cmap(
    df_day['color'], colormap)
# colors

I had to play with the "radius_min_pixels" as the line became really thin so I increased the size a bit:

In [37]:
m = leafmap.Map(height=600)
m.add_basemap('CartoDB.Positron')
data = df_day
m.add_gdf(data, get_fill_color = colors, auto_highlight = True, pickable = True, radius_min_pixels=1.2)
# m # PLEASE UNCOMMENT FOR VIEWING

Save the script to a html file:

In [38]:
m.to_html('day_cattle.html')

If you are in Google Colab, you can just download this on your local computer and open this in any browser. You should see something like this:

## Analysis for the night:

In [39]:
df_night.head()

,event-id,visible,timestamp,location-l,location_1,comments,ground-spe,heading,height-abo,study-spec,tag-tech-s,sensor-typ,individual,tag-local-,individu_1,study-name,geometry
98956,5171087856,True,2009-03-12 18:00:00.000,15.088900,11.146467,night,2.235200,80.0,316.0776,22-3,0,gps,Bos taurus,cattle2120309night,cattle2120309night,Daily grazing movements of cattle in the Far N...,POINT (15.0889 11.14647)
98957,5171087857,True,2009-03-12 18:00:00.000,15.088950,11.146483,night,0.044704,270.0,316.0776,6-38,1,gps,Bos taurus,cattle2120309night,cattle2120309night,Daily grazing movements of cattle in the Far N...,POINT (15.08895 11.14648)
98958,5171087858,True,2009-03-12 18:00:00.000,15.088933,11.146483,night,0.447040,71.0,317.9064,6-4,2,gps,Bos taurus,cattle2120309night,cattle2120309night,Daily grazing movements of cattle in the Far N...,POINT (15.08893 11.14648)
98959,5171087859,True,2009-03-12 18:00:00.000,15.088950,11.146483,night,0.581152,0.0,317.9064,6-3,3,gps,Bos taurus,cattle2120309night,cattle2120309night,Daily grazing movements of cattle in the Far N...,POINT (15.08895 11.14648)
98960,5171087860,True,2009-03-12 18:01:00.000,15.088950,11.146500,night,0.044704,316.0,317.9064,3-13,4,gps,Bos taurus,cattle2120309night,cattle2120309night,Daily grazing movements of cattle in the Far N...,POINT (15.08895 11.1465)


In [40]:
print(f'There are {len(counts_cattle_night)} cattle observed in the night')

There are 12 cattle observed in the night


Which cattle?

In [42]:
unique_values_night = df_night['tag-local-'].unique()
print(unique_values_night)

['cattle2120309night' 'cattle2130309night' 'cattle3110309night'
 'cattle3120309night' 'cattle3230309night' 'cattle3240309night'
 'cattle3250309night' 'cattle4110309night' 'cattle4160309night'
 'cattle4230309night' 'cattle4240309night' 'cattle4250309night']


What I wondered at this point is whether the same cattle is tagged during the day or the night as the tags did not seem to match. This is something to look into later and read the original paper of Moritz et al. Next, I follow the same process as above and create a unique "color" column:

In [43]:
df_night = df_night[['timestamp', 'tag-local-', 'geometry']]
df_night.head()

,timestamp,tag-local-,geometry
98956,2009-03-12 18:00:00.000,cattle2120309night,POINT (15.0889 11.14647)
98957,2009-03-12 18:00:00.000,cattle2120309night,POINT (15.08895 11.14648)
98958,2009-03-12 18:00:00.000,cattle2120309night,POINT (15.08893 11.14648)
98959,2009-03-12 18:00:00.000,cattle2120309night,POINT (15.08895 11.14648)
98960,2009-03-12 18:01:00.000,cattle2120309night,POINT (15.08895 11.1465)


Here, we need to assing colours again. I take the same range of colors (although the cattle may potentially differ). I am yet to figure this out though.

In [44]:
unique_values_night = df_night['tag-local-'].unique()
print(unique_values_night)

['cattle2120309night' 'cattle2130309night' 'cattle3110309night'
 'cattle3120309night' 'cattle3230309night' 'cattle3240309night'
 'cattle3250309night' 'cattle4110309night' 'cattle4160309night'
 'cattle4230309night' 'cattle4240309night' 'cattle4250309night']


In [45]:
def categorise_night(tag):
  for i in range(len(unique_values_night)):
    if tag == unique_values_night[i]:
      return i

In [46]:
# Apply the function to the Age column using the apply() function which I also name "color"
df_night['color'] = df_night['tag-local-'].apply(categorise_night)

In [47]:
df_night.color

,color
98956,0
98957,0
98958,0
98959,0
98960,0
...,...
153665,11
153666,11
153667,11
153668,11


In [48]:
df_night['color'].value_counts()

,count
color,
5,2322
8,2000
9,1807
4,1793
2,1661
10,1540
1,1473
6,1420
0,1392


In [49]:
df_night['color'] = df_night['color'].astype(int)

Take the same color map as above although the cows may differ (to figure out!):

In [50]:
colormap_night = {
  0: [26, 91, 171],
  1: [53, 130, 33],
  2: [167, 210, 130],
  3: [135, 209, 158],
  4: [255, 219, 92] ,
  5: [238, 207, 168],
  6: [237, 2, 42],
  7: [237, 233, 228],
  8: [242, 250, 255],
  9: [200, 200, 200],
  10: [65, 155, 223],
  11: [57, 125, 73],
}

In [51]:
len(colormap_night)

12

In [52]:
colors = lonboard.colormap.apply_categorical_cmap(df_night['color'], colormap_night)
colors

array([[ 26,  91, 171],
       [ 26,  91, 171],
       [ 26,  91, 171],
       ...,
       [ 57, 125,  73],
       [ 57, 125,  73],
       [ 57, 125,  73]], dtype=uint8)

Since these are night observations, let us have a dark background map:

In [56]:
m_night = leafmap.Map(height=600)
m_night.add_basemap('CartoDB.DarkMatter')
data = df_night
m_night.add_gdf(data, get_fill_color = colors, auto_highlight = True, pickable = True, radius_min_pixels=1.2)
# m_night # PLEASE UNCOMMENT FOR VIEWING. Otherwise, if I upload the program in Github it will crash.

In [57]:
m_night.to_html('night_cattle.html')

## To do/figure out:

- Are there layers like in GEEMap or Leafmap (or even in Lonboard) that one can switch on and off. Have a separate layer for the day and one for the night.

- Clean up the data more. As from the article of van Breugel, for one time stamp there are several movements/latitude and longitude. The analysis above is just using the raw data.

## References

[1] Moritz M. 2018. Data from: An integrated approach to modeling grazing pressure in pastoral systems: the case of the Logone Floodplain (Cameroon). Movebank Data Repository. [https://doi.org/10.5441/001/1.j682ds56](https://www.doi.org/10.5441/001/1.j682ds56)

[2] Moritz M, Soma E, Scholte P, Xiao N, Taylor L, Juran T, Kari S. 2010. An integrated approach to modeling grazing pressure in pastoral systems: the case of the Logone Floodplain (Cameroon). Hum Ecol. 38(6):775 [https://doi.org/10.1007/s10745-010-9361-z](https://doi.org/10.1007/s10745-010-9361-z)

[3] Paulo van Breugel, Playing with animal tracking data in QGIS. https://ecodiv.earth/post/animal_tracking-data/#ref-moritz2018

[4] Gandhi, U., course on [Mapping & Data Visualisation with Python](https://courses.spatialthoughts.com/python-dataviz.html#installation-and-setting-up-the-environment)

[5] Gandhi, U., [YouTube videos](https://www.youtube.com/playlist?list=PLppGmFLhQ1HLzGl8auwYkdUMu_z0Hz7G6) on Mapping and Data Visualization with Python Course

[6] Quisheng Wu, [Youtube videos ](https://www.youtube.com/watch?v=HE9jKZM6QhU&list=PLAxJ4-o7ZoPfb18kNe2luWX9xKg1233i9&index=18) on Intro to GIS programming.

[7] Quisheng, Wu, https://geemap.org/notebooks/115_land_cover/